### Strategy

1. Build classifier for homes based upon Zestimate under, normal, and overprediction
2. Train different regression models on each of the three classifications
3. Use classification model to predict classification of test properties
4. Run augmented test data through three separate models
5. Somehow intergrate time-dependent modeling

### Load Libraries

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing
import sklearn.ensemble

### Load Data

In [23]:
path = "Data/train.csv"
train = pd.read_csv(path)

path_2 ="Data/test.csv"
test = pd.read_csv(path_2)

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (23,33,35,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Remove Train Features Missing 90% of Data

In [24]:
missing = train.isnull().sum(axis=0).reset_index()
missing.columns = ['column_name', 'missing_count']
missing = missing.loc[missing['missing_count']>=0]
#missing = missing.sort_values(by='missing_count')
drop = list(missing[missing.missing_count/train.shape[0] >= .90].column_name.values)

In [25]:
train = train.drop(drop, 1)
test = test.drop(drop, 1)

### Label Train Data as Over(2), Under(1) or Normal(0) Logerror

In [26]:
#Need to figure out optimum percentage split
overpreds = np.percentile(train.logerror.values, 90)
underpreds = np.percentile(train.logerror.values, 10)

In [27]:
train['logerr_class'] = [2 if x >= overpreds else 1 if x <= underpreds else 0 for x in train['logerror']]

In [30]:
train.head()

,Unnamed: 0,parcelid,logerror,transactiondate,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerr_class
0,0,11016594,0.0276,2016-01-01,1.0,2.0,3.0,4.0,2.0,1684.0,...,1.0,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,6.037107e+13,0
1,1,14366692,-0.1684,2016-01-01,NaN,3.5,4.0,NaN,3.5,2263.0,...,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,1
2,2,12098116,-0.0040,2016-01-01,1.0,3.0,2.0,4.0,3.0,2217.0,...,1.0,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,6.037464e+13,0
3,3,12643413,0.0218,2016-01-02,1.0,2.0,2.0,4.0,2.0,839.0,...,1.0,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,6.037296e+13,0
4,4,14432541,-0.0050,2016-01-02,NaN,2.5,4.0,NaN,2.5,2283.0,...,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,6.059042e+13,0


In [53]:
example = train.drop(train.columns[0:3],1)

In [54]:
dtype_df = example.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()
drop_object = list(dtype_df[dtype_df['Column Type'] == "object"].Count.values)

In [55]:
X = example.drop("logerr_class", 1 )
X = X.drop(drop_object, 1)

y = example.logerr_class

In [56]:
imp_median = sklearn.preprocessing.Imputer(missing_values='NaN', strategy='median', axis=0)
X = imp_median.fit_transform(X)
#X = pd.DataFrame(X, columns = train.columns)

In [57]:
gbc = sklearn.ensemble.GradientBoostingClassifier(learning_rate = 0.1, n_estimators=100, max_depth = 3, random_state = 100)
gbc_scores = sklearn.model_selection.cross_val_score(gbc, X, y, cv=10, scoring='accuracy')
#gbc = gbc.fit(X,y)
#train_gbc_acc = gbc.score(X,y)
cv_gbc_acc = gbc_scores.mean()

In [58]:
cv_gbc_acc

0.79855994242571104